# How to evaluate text clustering algorithms with BCubed?

The B-cubed method is described in detail in:

[1] Bagga, Amit and Breck Baldwin. 1998. Algorithms for scoring coreference chains. In Proceedings of the First International Conference on Language Resources and Evaluation Workshop on Linguistic Coreference.

The examples are taken from this paper:

[2] Amigó, E., Gonzalo, J., Artiles, J. et al. A comparison of extrinsic clustering evaluation metrics based on formal constraints. Inf Retrieval 12, 461–486 (2009). https://doi.org/10.1007/s10791-008-9066-8

In the paper the authors analyzed a wide range of metrics and showed that only BCubed satisfies all formal constraints. However, it is not suitable for overlapping clustering evaluation. 

## How is BCubed Precision and BCubed Recall calculated?

Precision represents how many items in the same cluster belong to its category. 
The Recall associated to one item represents how many items from its category appear in its cluster. [2]

<img src="1.jpg">

## How to interpret Precision and Recall in BCubed? 

High BCubed __Recall__: Most related items are in a cluster

High BCubed __Precision__: No noisy items in a cluster [2]

## How to easy calculate Precision and Recall?

$$Recall (R) = \frac{\text{number of correct elements in the cluster}^{2}}{\text{number of elements in the cluster}}$$

$$Precision (P) = \frac{\text{number of correct elements in the cluster}^{2}}{\text{number of these elements in all cluster}}$$

$$F(BCubed) = \frac{2 * P * R}{P + R}$$


## Let's calculate the following (left clustered) example [2]

<img src="3.png" width="200">

$$Recall (R) = \frac{\frac{4*4}{5} + \frac{1}{5}+ \frac{2*2}{6}+ \frac{3*1}{1}+ \frac{4*4}{6}}{14} = 0.69$$


$$Precision (P) = \frac{\frac{4*4}{4} + \frac{1}{3}+ \frac{2*2}{3}+ \frac{3*1}{7}+ \frac{4*4}{7}}{14} = 0.59 $$

$$F(BCubed) = \frac{2 * 0.69 * 0.59}{0.69 + 0.59} =  = 0.63$$

In [1]:
%pip install bcubed

Note: you may need to restart the kernel to use updated packages.


In [2]:
import bcubed

"""
   Compute Bcubed Precision, Recall and F-Score 
   clustering_dict: dictionary representing clustering output
   gold_standard_dict: ground-truth dictionary
   Format for both dictionaries: {item: set of assigned clusters/real categories}
"""

def bcubed_compute(clustering_dict, gold_standard_dict):
    precision = bcubed.precision(clustering_dict, gold_standard_dict)
    recall = bcubed.recall(clustering_dict, gold_standard_dict)
    fscore = bcubed.fscore(precision, recall)
    print("precision={:.2f}, recall={:.2f}, fscore={:.2f}".format(precision, recall, fscore))

In [15]:
# example ground-truth data (ldict)
ground_truth = {
    "item1": set(["black", "black", "black"]),
    "item2": set(["gray", "gray", "gray"]),
    "item3": set(["blue", "blue", "blue"]),
    "item4": set(["dashed", "dashed", "dashed"]),
}

# example clustering (cdict) in page 24, figure 16
clustering = {
    "item1": set(["black", "black", "gray"]),
    "item2": set(["black", "gray", "gray"]),
    "item3": set(["blue", "blue", "blue"]),
    "item4": set(["dashed", "dashed", "dashed"]),
}
bcubed_compute(clustering, ground_truth)

precision=0.62, recall=1.00, fscore=0.77


In [4]:
# example ground-truth data (ldict)
ground_truth = {
    "item1": set(["gray", "black"]),
    "item2": set(["gray", "black"]),
    "item3": set(["gray"]),
    "item4": set(["black"]),
    "item5": set(["black"]),
    "item6": set(["dashed"]),
    "item7": set(["dashed"]),
}

# example clustering (cdict) in page 24, figure 16
clustering = {
    "item1": set(["A", "B"]),
    "item2": set(["A", "B"]),
    "item3": set(["A"]),
    "item4": set(["B"]),
    "item5": set(["B"]),
    "item6": set(["C"]),
    "item7": set(["C"]),
}
bcubed_compute(clustering, ground_truth)

precision=1.00, recall=1.00, fscore=1.00


In [5]:
# example clustering (cdict) in page 24, figure 17
clustering = {
    "item1": set(["ADup", "A", "B"]),
    "item2": set(["ADup", "A", "B"]),
    "item3": set(["ADup", "A"]),
    "item4": set(["B"]),
    "item5": set(["B"]),
    "item6": set(["C"]),
    "item7": set(["C"]),
}
bcubed_compute(clustering, ground_truth)

precision=0.86, recall=1.00, fscore=0.93


In [8]:
ground_truth = {
    "item1": set(["1", "2", "3", "4", "5"]),
    "item2": set(["6", "7"]),
    "item3": set(["8", "9", "A", "B", "C"]),
}

clustering = {
    "item1": set(["1", "2", "3", "4", "5", "8", "9", "A", "B", "C"]),
    "item2": set(["6", "7"]),
}

bcubed_compute(clustering, ground_truth)

precision=0.75, recall=1.00, fscore=0.86


Source:

http://e-spacio.uned.es/fez/eserv/bibliuned:DptoLSI-ETSI-MA2VICMR-1090/Documento.pdf

http://www.cs.cmu.edu/~yimengz/papers/Coreference_survey.pdf

https://www.uni-weimar.de/medien/webis/events/pan-16/pan16-papers-final/pan16-author-identification/sari16-notebook.pdf 